In [1]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3", temperature=0) #deterministico

In [2]:
llm.invoke("What is LangGraph?")

'LangGraph is a graph-based language model that represents natural language as a graph structure. It\'s a novel approach to modeling language, which can be used for various NLP tasks such as text classification, sentiment analysis, and machine translation.\n\nIn traditional language models, words are represented as vectors in a high-dimensional space (e.g., word embeddings like Word2Vec or GloVe). LangGraph takes a different approach by representing sentences or texts as graphs, where nodes represent words or subwords, and edges capture the relationships between them.\n\nThe graph structure is designed to capture various aspects of language, such as:\n\n1. **Word order**: The order in which words appear in a sentence.\n2. **Dependency relations**: The grammatical relationships between words, like subject-verb-object dependencies.\n3. **Semantic roles**: The roles played by entities in a sentence (e.g., "agent" or "patient").\n4. **Coreference**: The relationships between pronouns and t

In [16]:
from langchain_community.document_loaders import WebBaseLoader
# Lista de URLs
urls = [
    "https://www.rosario.gob.ar/inicio/licencia-de-conducir",
    "https://www.rosario.gob.ar/inicio/duplicado-de-licencia-de-conducir",
    "https://www.rosario.gob.ar/inicio/nueva-licencia-de-conducir",
    "https://www.rosario.gob.ar/inicio/renovacion-de-licencia-de-conducir",
    "https://www.rosario.gob.ar/inicio/renovacion-de-licencia-de-conducir-con-ampliacion"
]


loader = WebBaseLoader(web_path=urls)
docs = loader.load()

In [18]:
docs[1].page_content

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDuplicado de licencia de conducir | rosario.gob.ar \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nIr al contenido principal\n\n\n\n\n\n\n\n\n              rosario.gob.ar\n\n\n\n\nBuscar en rosario.gob.ar\n\n\n\n\n\n\n\n\nServicios\n\n\nPerfil digital\n\n\nConsultas y reclamos\n\n\nDenuncias\n\n\nActividad económica y trabajo\n\n\nAmbiente y mantenimiento urbano\n\n\nCementerios\n\n\nCiudad y turismo\n\n\nCultura y educación\n\n\nDeportes y recreación\n\n\nImpuestos y tasas\n\n\nMovilidad y tránsito\n\n\nParticipación, transparencia e información pública\n\n\nPersonas y Comunidades\n\n\nSalud\n\n\nSalud animal\n\n\nVivienda y construcción\n\n\n\n\nGobierno\n\n\nAutoridades y secretarías\n\n\nPersonal municipal\n\n\nPresupuesto\n\n\nModernización\n\n\nInfomapa\n\n\nInformes y datos\n\n\nCompras y licitaciones\n\n\nNormativa\n\n\nNoticias\n\n\n\n\nAgenda\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\

In [19]:
len(docs[0].page_content)

10149

In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,    
    add_start_index=True,
)

In [21]:
all_splits = text_splitter.split_documents(docs)

In [22]:
len(all_splits)

72

In [8]:
all_splits[1].page_content

'Buscar\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nServicios\n\n\nPerfil digital\n\n\nConsultas y reclamos\n\n\nDenuncias\n\n\nActividad económica y trabajo\n\n\nAmbiente y mantenimiento urbano\n\n\nCementerios\n\n\nCiudad y turismo\n\n\nCultura y educación\n\n\nDeportes y recreación\n\n\nImpuestos y tasas\n\n\nMovilidad y tránsito\n\n\nParticipación, transparencia e información pública\n\n\nPersonas y Comunidades\n\n\nSalud\n\n\nSalud animal\n\n\nVivienda y construcción\n\n\n\n\nGobierno\n\n\nAutoridades y secretarías\n\n\nPersonal municipal\n\n\nPresupuesto\n\n\nModernización\n\n\nInfomapa\n\n\nInformes y datos\n\n\nCompras y licitaciones\n\n\nNormativa\n\n\nNoticias\n\n\n\n\nAgenda\n\n\n\n\n\n\n\nInformación importante\n\n\n\n\n\n\n\n\nInicio\n\n\nLicencia de conducir\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \nLa licencia nacional de conducir es un documento único que habilita legalmente a un/a ciudadano/a a conducir uno o más tipos de vehículos. Las clases se incluyen en una única crede

In [9]:
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model = "nomic-embed-text"
)

In [23]:
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(
    documents = all_splits, 
    embedding = embedding
    )

In [24]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}
    )

In [13]:
retriever.get_relevant_documents(query="Que es el Cenat?")

/tmp/ipykernel_3935/3014629624.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(query="Que es el Cenat?")


[Document(metadata={'language': 'es', 'source': 'https://www.rosario.gob.ar/inicio/licencia-de-conducir', 'start_index': 3269, 'title': 'Licencia de conducir | rosario.gob.ar '}, page_content='Cómo pagar boleta CeNAT\n\n\n\n1) CeNAT (Certificado Nacional de Antecedentes de Tránsito)\nDe manera virtual\n\nTarjeta de débito: desde Plus Pagos\xa0o\xa0escaneando el QR de la primera hoja de la boleta.\nMercado Pago: escaneando el código de barras de la primera hoja de la boleta.\nMODO: escaneando el\xa0QR de la primera hoja de la boleta.\n\nDe manera presencial\n\nEn cualquier sucursal de Munipagos:\xa0presentar la segunda hoja de la boleta.\nEn cualquier sucursal de Santa Fe Servicios: presentar la segunda hoja de la boleta.\nEn cualquier sucursal de\xa0Pago fácil:\xa0presentar la primera hoja de la boleta.\nEn el Banco Nación: presentar\xa0la primera hoja de la boleta.\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\nCómo pagar boleta sellado municipal\n\n\n\nTené en cuenta que la boleta es una sola

In [12]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

/home/gcasado0/proyectos/chatbot/chatbot/lib/python3.12/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [13]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [25]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
rag_chain = ( 
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
    )

In [27]:
rag_chain.invoke("extravie mi licencia que tengo que llevar para hacer el duplicado? responder en español")

'Para obtener el duplicado de tu licencia, debes descargar y completar la Declaración jurada por extravío.'